In [1]:
from pretty_midi import PrettyMIDI, instrument_name_to_program, Instrument, note_name_to_number, Note
from os import system
import pandas as pd
import pickle
import torch

In [2]:
trainset = pickle.load(open("new_trainset.pkl", "rb"))
testset = pickle.load(open("new_testset.pkl", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'new_trainset.pkl'

In [4]:
len(trainset)

709

In [4]:
KEYS = ("C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B") 

def synthesize(transition_progress, next_chords, original_csv_path, cleaned_csv_path, original_midi_path, output_midi_path, transition_progress_threshold=0.05, instrument="String Ensemble 1", velocity=50, group=3, melody_only=True, to_mp3=False):
    midi = PrettyMIDI(original_midi_path)
    if melody_only:
        midi.instruments = [midi.instruments[0]]
    program = instrument_name_to_program(instrument)
    accompany = Instrument(program=program, name="accompany")

    cleaned_df = pd.read_csv(cleaned_csv_path, index_col=0)
    last_start_index = int(cleaned_df.index[0])
    last_index = last_start_index
    last_total_duration = 0
    this_chord = next_chords[0]
    original_df = pd.read_csv(original_csv_path, index_col=0)
    for i, (transition, next_chord) in enumerate(zip(transition_progress[1:], next_chords[1:])):
        index = int(cleaned_df.index[i]) # start index of the current chord
        if index - last_index <= 1:
            last_total_duration += original_df.loc[index, "duration"]
        if transition < transition_progress_threshold or index - last_index > 1 or i == len(transition_progress) - 2:
            start_time = original_df.loc[last_start_index, "time"]
            #print(start_time, last_start_index, last_total_duration)
            for bit, key in zip(this_chord, KEYS):
                if bit:
                    note_number = note_name_to_number(f"{key}{group}")
                    note = Note(velocity=velocity, pitch=note_number, start=start_time, end=start_time+last_total_duration)
                    accompany.notes.append(note)
            if i < len(transition_progress) - 2:
                if transition < transition_progress_threshold:
                    this_chord = next_chord
                    last_total_duration = 0
                    last_start_index = int(cleaned_df.index[i + 1])
                else:
                    last_total_duration = original_df.loc[index, "duration"]
                    last_start_index = int(cleaned_df.index[i])
                
                
        last_index = index
    midi.instruments.append(accompany)
    midi.write(output_midi_path)
    if to_mp3:
        system(f"Musescore4 {output_midi_path} -o {output_midi_path.replace('mid', 'mp3')}")

In [45]:
idx, transition_progress, next_chords, melody_reprs = trainset[0]
len(transition_progress), len(next_chords)
synthesize(transition_progress, next_chords, f"POP909/POP909/{idx}/melody_chord_1_beat.csv", f"POP909/POP909/{idx}/melody_chord_1_beat_clean.csv", f"POP909/POP909/{idx}/{idx}.mid", f"test.mid")


In [13]:
idx = "001"
predicted = torch.load(f"song_{idx}_regularized.pt", map_location="cpu")[0,1:,:12]
original_df = pd.read_csv(f"POP909/POP909/{idx}/melody_chord_1_beat.csv", index_col=0)
cleaned_df = pd.read_csv(f"POP909/POP909/{idx}/melody_chord_1_beat_clean.csv", index_col=0)
original_midi_path = f"POP909/POP909/{idx}/{idx}.mid"
midi = PrettyMIDI(original_midi_path)
midi.instruments = [midi.instruments[0]]
program = instrument_name_to_program("String Ensemble 1")
accompany = Instrument(program=program, name="accompany")
last_chord = None
this_duration = 0
last_start_time = 0
last_index = -1
for i in range(len(predicted)):
    index = int(cleaned_df.index[i])
    start_time = original_df.loc[index, "time"]
    duration = original_df.loc[index, "duration"]
    predicted_chord = predicted[i]
    if last_chord is None or (predicted_chord != last_chord).any() or index - last_index > 1:
        if last_chord is not None:
            for bit, key in zip(last_chord, KEYS):
                if bit:
                    note_number = note_name_to_number(f"{key}{3}")
                    note = Note(velocity=50, pitch=note_number, start=last_start_time, end=last_start_time+this_duration)
                    accompany.notes.append(note)
        last_chord = predicted_chord
        last_start_time = start_time
        this_duration = duration
    else:
        this_duration += duration
    last_index = index
midi.instruments.append(accompany)
midi.write(f"test_{idx}_music103.mid")

In [5]:
def synthesize2(chord_reprs, original_csv_path, original_midi_path, output_midi_path, instrument="String Ensemble 1", velocity=50, group=3, melody_only=True, to_mp3=False):
    midi = PrettyMIDI(original_midi_path)
    if melody_only:
        midi.instruments = [midi.instruments[0]]
    program = instrument_name_to_program(instrument)
    accompany = Instrument(program=program, name="accompany")
    original_df = pd.read_csv(original_csv_path, index_col=0)
    beat_count = 0
    for chord_repr in chord_reprs[1:]:
        this_chord = chord_repr[:-1]
        last_length = chord_repr[-1]
        start_time = original_df.loc[beat_count, "time"]
        end_beat = beat_count + last_length - 1
        if end_beat >= len(original_df):
            end_beat = len(original_df) - 1
        end_time = original_df.loc[end_beat, "time"] + original_df.loc[end_beat, "duration"]
        for bit, key in zip(this_chord, KEYS):
            if bit:
                note_number = note_name_to_number(f"{key}{group}")
                note = Note(velocity=velocity, pitch=note_number, start=start_time, end=end_time)
                accompany.notes.append(note)
        beat_count += last_length
    midi.instruments.append(accompany)
    midi.write(output_midi_path)
    if to_mp3:
        system(f"Musescore4 {output_midi_path} -o {output_midi_path.replace('mid', 'mp3')}")

In [156]:

predicted = torch.load(f"song_{idx}_regularized.pt", map_location="cpu")[0,1:,:]

In [16]:
idx = "024"
predicted = torch.load(f"song_{idx}_regularized.pt", map_location="cpu")[0,1:,:]
original_csv_path = f"POP909/POP909/{idx}/melody_chord_1_beat.csv"
original_midi_path = f"POP909/POP909/{idx}/{idx}.mid"
midi = PrettyMIDI(original_midi_path)
midi.instruments = [midi.instruments[0]]
program = instrument_name_to_program("String Ensemble 1")
accompany = Instrument(program=program, name="accompany")
original_df = pd.read_csv(original_csv_path, index_col=0)
beat_count = 0
for repr in predicted:
    this_chord = repr[:12]
    last_length = torch.argmax(repr[12:]).item()
    print(last_length)
    start_time = original_df.loc[beat_count, "time"]
    end_beat = beat_count + last_length - 1
    if end_beat >= len(original_df):
        end_beat = len(original_df) - 1
    end_time = original_df.loc[end_beat, "time"] + original_df.loc[end_beat, "duration"]
    for bit, key in zip(this_chord, KEYS):
        if bit:
            note_number = note_name_to_number(f"{key}{3}")
            note = Note(velocity=50, pitch=note_number, start=start_time, end=end_time)
            accompany.notes.append(note)
    beat_count += last_length
    if beat_count >= len(original_df):
        break
midi.instruments.append(accompany)
midi.write(f"test_{idx}_music103.mid")

2
2
2
2
2
2
2
4
4
4
4
2
2
2
2
2
2
2
4
2
2
2
2
2
2
2
2
4
2
2
2
2
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
4
4
4
4
4
2
2
2
2
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
6
5
0
0


In [138]:
trainset = pickle.load(open("new_trainset2.pkl", "rb"))

In [139]:
chord_lengths = []
max_length = 0
for idx, melody_reprs, chord_reprs in trainset:
    for i, repr in enumerate(chord_reprs[1:]):
        if repr[-1] > 20:
            print(idx, i, repr[-1], repr)
            max_length = repr[-1]
        chord_lengths.append(repr[-1])


In [140]:
import numpy as np
np.mean(chord_lengths), np.std(chord_lengths)

(2.7703324273468533, 1.535275970530481)

In [155]:
idx, melody_reprs, chord_reprs = trainset[521]
print(idx)
synthesize2(chord_reprs, f"POP909/POP909/{idx}/melody_chord_1_beat.csv", f"POP909/POP909/{idx}/{idx}.mid", f"test.mid")

673


In [31]:
predicted = torch.load("song_test_music103.pt", map_location="cpu")
for idx_, chord_reprs_ in predicted:
    idx = idx_[0]
    chord_reprs = chord_reprs_[0]
    print(idx)
    original_df = pd.read_csv(f"POP909/POP909/{idx}/melody_chord_1_beat.csv", index_col=0)
    original_midi_path = f"POP909/POP909/{idx}/{idx}.mid"
    midi = PrettyMIDI(original_midi_path)
    midi.instruments = [midi.instruments[0]]
    program = instrument_name_to_program("String Ensemble 1")
    accompany = Instrument(program=program, name="accompany")
    old_chord = None
    for i, chord in enumerate(chord_reprs):
        beat = original_df.loc[i, "time"]
        duration = original_df.loc[i, "duration"]
        if (chord != old_chord) or i == len(chord_reprs) - 1:
            if old_chord:
                for bit, key in zip(old_chord, KEYS):
                    if bit:
                        note_number = note_name_to_number(f"{key}{3}")
                        note = Note(velocity=50, pitch=note_number, start=old_beat, end=old_beat+total_duration)
                        accompany.notes.append(note)
            old_beat = beat
            total_duration = duration
            old_chord = chord
        else:
            total_duration += duration
    midi.instruments.append(accompany)
    midi.write(f"test_{idx}_music103.mid")

639


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [21]:
chord_reprs

tensor([[[0, 0, 1,  ..., 1, 0, 0],
         [0, 1, 1,  ..., 1, 0, 1],
         [0, 1, 1,  ..., 1, 0, 0],
         ...,
         [0, 1, 1,  ..., 1, 0, 1],
         [0, 0, 1,  ..., 1, 0, 0],
         [0, 1, 1,  ..., 1, 0, 1]]])